This notebook uses taxon names from ITIS matches, looks for those names from the NatureServe species services, and returns key/value pairs for caching in the Taxonomic Information Registry and using in our systems.

In [1]:
import requests
from IPython.display import display
from bis import natureserve
from bis import tir
from bis2 import gc2
from bis2 import natureserve as natureservekeys

### Get data to process
Right now, we run this against ITIS names where the NatureServe information is null. In future, we need to work out how we run this periodically to refresh the cache and what we do with the previous data returned from the service.

In [2]:
q_itisNames = "SELECT id, \
    itis->'nameWInd' AS namewind, \
    itis->'nameWOInd' AS namewoind \
    FROM tir.tir \
    WHERE natureserve IS NULL AND itis->'itisMatchMethod' NOT LIKE 'NotMatched%' \
    LIMIT 10"
itisNames = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_itisNames).json()


### Loop through the names and run the functions
We set up a local record to house everything, run the functions, and update results in the TIR. If necessary and applicable, we try both the nameWOInd and nameWInd values from ITIS.

In [3]:
for feature in itisNames["features"]:
    thisRecord = {}
    thisRecord["id"] = feature["properties"]["id"]
    thisRecord["nameWOInd"] = feature["properties"]["namewoind"]
    thisRecord["nameWInd"] = feature["properties"]["namewind"]
    
    # Try to find a NatureServe ID with nameWOInd
    thisRecord["elementGlobalID"] = natureserve.queryNatureServeID(thisRecord["nameWOInd"])
    
    # Try to find a NatureServe ID with nameWInd if possible
    if thisRecord["elementGlobalID"] == "none" and thisRecord["nameWInd"] != thisRecord["nameWOInd"]:
        thisRecord["elementGlobalID"] = natureserve.queryNatureServeID(natureservekeys.speciesAPI(),thisRecord["nameWInd"])

    # Run the function to query and pacage NatureServe key/value pairs
    thisRecord["natureServePairs"] = natureserve.packageNatureServePairs(natureservekeys.speciesAPI(),thisRecord["elementGlobalID"])

    # Display the record, cache results, and show query status
    display (thisRecord)
#    print (gc2.sqlAPI("DataDistillery","BCB"),tir.cacheToTIR(thisRecord["id"],"natureserve",thisRecord["natureServePairs"]))

{'elementGlobalID': 'ELEMENT_GLOBAL.2.104998',
 'id': 944,
 'nameWInd': 'Otus flammeolus',
 'nameWOInd': 'Otus flammeolus',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:24.852430","elementGlobalID"=>"ELEMENT_GLOBAL.2.104998","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"2016-04-09","usNationalStatusRankCode"=>"N4B","usNationalStatusLastReviewed"=>"1997-03-19","StateCode:Arizona"=>"S4","StateCode:California"=>"S2S4","StateCode:Colorado"=>"S4","StateCode:Idaho"=>"S3B","StateCode:Montana"=>"S3B","StateCode:Navajo Nation"=>"S4B","StateCode:Nevada"=>"S4B","StateCode:New Mexico"=>"S3B,S3N","StateCode:Oregon"=>"S3B","StateCode:South Dakota"=>"S1B","StateCode:Texas"=>"S3B","StateCode:Utah"=>"S3S4B","StateCode:Washington"=>"S3B","StateCode:Wyoming"=>"S1?"'}

{'elementGlobalID': 'none',
 'id': 649,
 'nameWInd': 'Catostomus',
 'nameWOInd': 'Catostomus',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:28.273311","elementGlobalID"=>"none","status"=>"Not Found"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.104930',
 'id': 1141,
 'nameWInd': 'Spea intermontana',
 'nameWOInd': 'Spea intermontana',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:30.346633","elementGlobalID"=>"ELEMENT_GLOBAL.2.104930","GlobalStatusRank"=>"G5","roundedGlobalStatusRankDescription"=>"Secure","globalStatusLastReviewed"=>"2016-08-02","usNationalStatusRankCode"=>"N5","usNationalStatusLastReviewed"=>"1996-11-05","StateCode:Arizona"=>"S3","StateCode:California"=>"SNR","StateCode:Colorado"=>"S3","StateCode:Idaho"=>"S4","StateCode:Nevada"=>"S4","StateCode:Oregon"=>"S5","StateCode:Utah"=>"S5","StateCode:Washington"=>"S5","StateCode:Wyoming"=>"S3"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.106154',
 'id': 865,
 'nameWInd': 'Leucosticte australis',
 'nameWOInd': 'Leucosticte australis',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:34.511005","elementGlobalID"=>"ELEMENT_GLOBAL.2.106154","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"1996-12-04","usNationalStatusRankCode"=>"N4B,N4N","usNationalStatusLastReviewed"=>"1997-01-05","StateCode:Colorado"=>"S3B,S4N","StateCode:New Mexico"=>"S1B,S3N","StateCode:Wyoming"=>"S1"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.101454',
 'id': 868,
 'nameWInd': 'Lithobates pipiens',
 'nameWOInd': 'Lithobates pipiens',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:37.688797","elementGlobalID"=>"ELEMENT_GLOBAL.2.101454","GlobalStatusRank"=>"G5","roundedGlobalStatusRankDescription"=>"Secure","globalStatusLastReviewed"=>"2016-05-10","usNationalStatusRankCode"=>"N5","usNationalStatusLastReviewed"=>"1996-11-05","StateCode:Arizona"=>"S2","StateCode:California"=>"S2","StateCode:Colorado"=>"S3","StateCode:Connecticut"=>"S2","StateCode:Idaho"=>"S2","StateCode:Illinois"=>"S5","StateCode:Indiana"=>"S2","StateCode:Iowa"=>"S5","StateCode:Kentucky"=>"S3","StateCode:Maine"=>"S3","StateCode:Maryland"=>"S4","StateCode:Massachusetts"=>"S3S4","StateCode:Michigan"=>"S5","StateCode:Minnesota"=>"S4","StateCode:Missouri"=>"S2","StateCode:Montana"=>"S1,S4","StateCode:Navajo Nation"=>"S2","StateCode:Nebraska"=>"S5","StateCode:Nevada"=>"S2S3","StateCode:New Hampshire"=>"S3","StateCode:New Je

{'elementGlobalID': 'ELEMENT_GLOBAL.2.103108',
 'id': 827,
 'nameWInd': 'Hyla wrightorum',
 'nameWOInd': 'Hyla wrightorum',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:40.630596","elementGlobalID"=>"ELEMENT_GLOBAL.2.103108","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"2008-07-07","usNationalStatusRankCode"=>"N4","usNationalStatusLastReviewed"=>"2001-10-26","StateCode:Arizona"=>"S4","StateCode:New Mexico"=>"S3"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.105750',
 'id': 1071,
 'nameWInd': 'Smilisca fodiens',
 'nameWOInd': 'Smilisca fodiens',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:43.781090","elementGlobalID"=>"ELEMENT_GLOBAL.2.105750","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"2002-04-02","usNationalStatusRankCode"=>"N1N2","usNationalStatusLastReviewed"=>"1996-11-01","StateCode:Arizona"=>"S2"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.104784',
 'id': 1197,
 'nameWInd': 'Tyrannus crassirostris',
 'nameWOInd': 'Tyrannus crassirostris',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:46.500154","elementGlobalID"=>"ELEMENT_GLOBAL.2.104784","GlobalStatusRank"=>"G5","roundedGlobalStatusRankDescription"=>"Secure","globalStatusLastReviewed"=>"1996-12-02","usNationalStatusRankCode"=>"N2B","usNationalStatusLastReviewed"=>"1997-03-19","StateCode:Arizona"=>"S2","StateCode:New Mexico"=>"S1B,S1N","StateCode:Texas"=>"S1B"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.101291',
 'id': 798,
 'nameWInd': 'Gymnorhinus cyanocephalus',
 'nameWOInd': 'Gymnorhinus cyanocephalus',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:49.176811","elementGlobalID"=>"ELEMENT_GLOBAL.2.101291","GlobalStatusRank"=>"G5","roundedGlobalStatusRankDescription"=>"Secure","globalStatusLastReviewed"=>"1996-12-02","usNationalStatusRankCode"=>"N5","usNationalStatusLastReviewed"=>"1997-01-05","StateCode:Arizona"=>"S5","StateCode:California"=>"SNR","StateCode:Colorado"=>"S5","StateCode:Idaho"=>"S1","StateCode:Montana"=>"S3","StateCode:Navajo Nation"=>"S5","StateCode:Nebraska"=>"S3","StateCode:Nevada"=>"S3S4","StateCode:New Mexico"=>"S3B,S3N","StateCode:Oklahoma"=>"S2","StateCode:Oregon"=>"S3S4","StateCode:South Dakota"=>"S4B,S4N","StateCode:Texas"=>"SNA","StateCode:Utah"=>"S4","StateCode:Washington"=>"SNA","StateCode:Wyoming"=>"S5B,S5N"'}

{'elementGlobalID': 'ELEMENT_GLOBAL.2.106046',
 'id': 657,
 'nameWInd': 'Charadrius melodus',
 'nameWOInd': 'Charadrius melodus',
 'natureServePairs': '"cacheDate"=>"2017-06-22T13:47:53.481258","elementGlobalID"=>"ELEMENT_GLOBAL.2.106046","GlobalStatusRank"=>"G3","roundedGlobalStatusRankDescription"=>"Vulnerable","globalStatusLastReviewed"=>"2016-04-07","usNationalStatusRankCode"=>"N3B,N3N","usNationalStatusLastReviewed"=>"1997-01-05","StateCode:Alabama"=>"S1N","StateCode:Arkansas"=>"SNA","StateCode:Colorado"=>"S1B","StateCode:Connecticut"=>"S1B","StateCode:Delaware"=>"S1B","StateCode:Florida"=>"S2","StateCode:Georgia"=>"S1","StateCode:Illinois"=>"SH","StateCode:Indiana"=>"SXB","StateCode:Iowa"=>"S1B","StateCode:Kansas"=>"S1B,S2N","StateCode:Kentucky"=>"SNA","StateCode:Louisiana"=>"S2N","StateCode:Maine"=>"S2B","StateCode:Maryland"=>"S1B","StateCode:Massachusetts"=>"S2B","StateCode:Michigan"=>"S1","StateCode:Minnesota"=>"S1B","StateCode:Mississippi"=>"S2N","StateCode:Missouri"=>"SNA","